# MWS Secondary Target Selection

## MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV

Maintainers:
  - David Nidever (selection/catalog)
  - Ting Li
  - Andrew Cooper (secondary target code)

In [1]:
ls -l ../raw

total 1
lrwxrwxrwx 1 apcooper apcooper 104 Sep  1 07:35 allgiants_xm_gaia_4DESI_v0.fits -> /project/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_SV/1/raw/allgiants_xm_gaia_4DESI_v0.fits


{ DESCRIPTION NEEDED }

The selection criteria are:

???

In [2]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV/1/code'

In [3]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [4]:
target_class   = 'MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV'
input_filename = 'allgiants_xm_gaia_4DESI_v0.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [5]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [6]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV/1/raw/allgiants_xm_gaia_4DESI_v0.fits
Read 291411 rows


In [7]:
print('First 3 rows:')
print(d[0:3])
print(d.colnames)

First 3 rows:
      ra           dec      ... phot_bp_mean_mag phot_rp_mean_mag
------------- ------------- ... ---------------- ----------------
328.519773312 2.40651951768 ...          16.2657          15.0581
328.530788028 2.53707028993 ...          19.5638          18.0976
328.592711696 2.42055391399 ...          15.1983          14.0163
['ra', 'dec', 'source_id', 'pmra', 'pmdec', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag']


In [8]:
BRIGHT_TIME_BRIGHT_LIMIT = 16.0

print('Mag range: {:5.2f} < G < {:5.2f}'.format(d['phot_g_mean_mag'].min() ,d['phot_g_mean_mag'].max()))
print('Targets brighter than G = 16: {}/{} ({:3.2%})'.format((d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum(),
                                                   len(d),
                                                   (d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum()/len(d)
                                                  ))

verybright_time_targets = d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT

Mag range: 10.67 < G < 20.88
Targets brighter than G = 16: 75258/291411 (25.83%)


Adjust to secondary target data model:

In [9]:
output = d['ra','dec','source_id'][verybright_time_targets]

for cn_old, cn_new in zip(['ra','dec','source_id'],['RA','DEC','REF_ID']):
    output.rename_column(cn_old,cn_new)

output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [10]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

In [11]:
output[0:3]

RA,DEC,REF_ID,OVERRIDE,PMRA,PMDEC,REF_CAT,REF_EPOCH
float64,float64,int64,bool,float32,float32,str1,str6
328.519773312,2.40651951768,2693915706957072256,False,0.0,0.0,G,2015.5
328.592711696,2.42055391399,2693913237351341184,False,0.0,0.0,G,2015.5
328.630733623,2.43554189173,2693913306070816384,False,0.0,0.0,G,2015.5


Write output:

In [12]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 75258 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV/1/indata/MWS_SPECIAL_DDOGIANTS_VERYBRIGHT_SV.fits
